<a href="https://colab.research.google.com/github/Namtk214/Categorizing-Spam-Email/blob/main/Categorizing_SpamEmail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Import các thư viện cần thiết

In [ ]:
import string #Thư viện mở rộng của string
import nltk #Thư viện xử lý text
nltk.download('punkt')
nltk.download('stopwords')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2. Read Dataset

In [ ]:
DATASET_PATH = '/content/2cls_spam_text_cls.csv'
df = pd.read_csv(DATASET_PATH)
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
messages = df['Message'].values.tolist()
labels = df['Category'].values.tolist() # Chuyển về kiểu dữ liệu list

3. Chuẩn bị bộ dữ liệu

In [ ]:
def lowercase(text):
  return text.lower()


In [ ]:
def tokenize(text):
  return nltk.word_tokenize(text) # giống return text.split()


In [ ]:
def remove_stopwords(tokens):
  stopwords = nltk.corpus.stopwords.words('english')
  return [token for token in tokens if token not in stopwords] #bỏ các từ ko quá quan trọng trong tiếng Anh. VD: a, an the...



In [ ]:
def stemming(tokens):
  stemmer = nltk.stem.PorterStemmer()
  return [stemmer.stem(token) for token in tokens] #chuyển về dạng base của các từ. VD: went => go



In [ ]:
def preprocess(text):
  text = lowercase(text)
  tokens = tokenize(text)
  tokens = remove_stopwords(tokens)
  tokens = stemming(tokens)
  return tokens



In [ ]:
messages = [preprocess(message) for message in messages]
messages

[['go',
  'jurong',
  'point',
  ',',
  'crazi',
  '..',
  'avail',
  'bugi',
  'n',
  'great',
  'world',
  'la',
  'e',
  'buffet',
  '...',
  'cine',
  'got',
  'amor',
  'wat',
  '...'],
 ['ok', 'lar', '...', 'joke', 'wif', 'u', 'oni', '...'],
 ['free',
  'entri',
  '2',
  'wkli',
  'comp',
  'win',
  'fa',
  'cup',
  'final',
  'tkt',
  '21st',
  'may',
  '2005.',
  'text',
  'fa',
  '87121',
  'receiv',
  'entri',
  'question',
  '(',
  'std',
  'txt',
  'rate',
  ')',
  '&',
  'c',
  "'s",
  'appli',
  '08452810075over18',
  "'s"],
 ['u', 'dun', 'say', 'earli', 'hor', '...', 'u', 'c', 'alreadi', 'say', '...'],
 ['nah', "n't", 'think', 'goe', 'usf', ',', 'live', 'around', 'though'],
 ['freemsg',
  'hey',
  'darl',
  "'s",
  '3',
  'week',
  "'s",
  'word',
  'back',
  '!',
  "'d",
  'like',
  'fun',
  'still',
  '?',
  'tb',
  'ok',
  '!',
  'xxx',
  'std',
  'chg',
  'send',
  ',',
  '£1.50',
  'rcv'],
 ['even',
  'brother',
  'like',
  'speak',
  '.',
  'treat',
  'like',
  'ai

4. Bag of words (vectorize và tạo dictionary gồm các từ trong dataset)

In [ ]:
def create_dictionary(messages):
  dictionary = []
  for tokens in messages:
    for token in tokens:
      if token not in dictionary:
        dictionary.append(token)
  return dictionary


In [ ]:
def create_feature(tokens, dictionary):
  features = np.zeros(len(dictionary))
  for token in tokens:
    if token in dictionary:
      features[dictionary.index(token)] = 1
  return features

In [ ]:
dictionary = create_dictionary(messages)
x = np.array([create_feature(tokens, dictionary) for tokens in messages])
x
len(dictionary)

7997

5. Xây model

In [ ]:
le = LabelEncoder() #Đổi category thành dạng binary
y = le.fit_transform(labels)
y

array([0, 0, 1, ..., 0, 0, 0])

In [ ]:
VAL_SIZE = 0.2
TEST_SIZE = 0.125
SEED = 0
IS_SHUFFLE = True

# Split into training + validation and testing sets
x_train_val, x_test, y_train_val, y_test = train_test_split(
    x, y,
    test_size=TEST_SIZE,
    random_state=SEED,
    shuffle=IS_SHUFFLE
)

# Split training + validation into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(
    x_train_val, y_train_val,
    test_size=VAL_SIZE,
    random_state=SEED,
    shuffle=IS_SHUFFLE
)
#
#x_train.shape[0]

In [ ]:
%%time
model = GaussianNB() #Training các model dùng hàm....
print('Start training')
model.fit(x_train, y_train)
print('Completed')

Start training
Completed
CPU times: user 481 ms, sys: 399 ms, total: 880 ms
Wall time: 874 ms


In [ ]:
y_val_pred = model.predict(x_val)
y_test_pred = model.predict(x_test)
val_accuracy = accuracy_score(y_val, y_val_pred)
test_accurancy = accuracy_score(y_test, y_test_pred)
#

In [ ]:
def predict(test, model, dictionary):
  processed_test = preprocess(test)
  features = create_feature(processed_test, dictionary)
  features = np.array(features).reshape(1, -1)
  prediction_cls = le.inverse_transform(model.predict(features))
  return prediction_cls



In [ ]:
test = 'Job'
prediction_cls = predict(test, model, dictionary)
prediction_cls

array(['ham'], dtype='<U4')